In [4]:
import numpy as np
import os
import random
from tqdm.auto import tqdm
import cv2
import pyarrow.parquet as pq
import glob
import math
import cupy as cp
from einops import rearrange
import time as time
import matplotlib.pyplot as plt

In [34]:
file = pq.ParquetFile("./Data_Raw_Tau/DYToTauTau_M-10_13TeV_pythia8_2018_ext1/DYToTauTau_M-10_13TeV_pythia8_2018_ext1_0.parquet")
batch = file.iter_batches(2048)
batch = next(batch)
batch = batch.to_pandas()
im = cp.array(np.array(np.array(batch.iloc[:, 3].tolist()).tolist()).tolist())
meta = np.array(batch.iloc[:, 0])

im[im < 1.e-3] = 0 #Zero_suppression
im = rearrange(im, 'b c h w -> b h w c')
for _z in range(13):
	im[:,:,:,_z] = (im[:,:,:,_z] - im[:,:,:,_z].mean())/(im[:,:,:,_z].std())
	im[:,:,:,_z] = cp.clip(im[:,:,:,_z], a_min = 0, a_max = 500*im[:,:,:,_z].std(axis = (1,2))[:,None,None])
 
im = im.get()
img = im[0,:,:,:]
for _z in range(13):
	
	if (img[:,:,_z]==0).all() == False:
		img[:,:,_z] = 255*(img[:,:,_z])/(img[:,:,_z].max())



img = img.astype(np.uint8)
img = rearrange(img, 'h w c -> h ( w c )')

OSError: Couldn't deserialize thrift: bad allocation


In [29]:
img2 = plt.imread("./Data_extracted/DYToTauTau_M-10_13TeV_pythia8_2018_ext1/1.png")

In [30]:
img = img/255